In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load and preprocess dataset
data = pd.read_csv('128V_NISHA.csv')
data.columns = [
    'Vehicle_ID',
    'Time',
    'Position_X',
    'Position_Y',
    'Speed',
    'Acceleration',
    'Distance_to_Nearest_Vehicle',
    'Signal_Strength',
    'Content_Request_Size'
]

# Feature engineering
data['Velocity_Change'] = data['Speed'] - data['Acceleration']

X = data.drop('Content_Request_Size', axis=1)
y = data['Content_Request_Size']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# Reshape data for LSTM-GRU
X_train_scaled_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Define and train LSTM-GRU model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train_scaled_reshaped.shape[1], X_train_scaled_reshaped.shape[2])),
    GRU(50),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
history = model.fit(X_train_scaled_reshaped, y_train_scaled, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
loss = model.evaluate(X_test_scaled_reshaped, y_test_scaled, verbose=1)
y_pred_scaled = model.predict(X_test_scaled_reshaped)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test_scaled)

# Model testing and validation
mse = mean_squared_error(y_test_actual, y_pred)
mae = mean_absolute_error(y_test_actual, y_pred)
print(f'Model Mean Squared Error: {mse}')
print(f'Model Mean Absolute Error: {mae}')

# Zone Construction and Clustering
kmeans = KMeans(n_clusters=5, random_state=42)
data['Zone'] = kmeans.fit_predict(data[['Position_X', 'Position_Y']])
zone_properties = data.groupby('Zone').agg({
    'Position_X': ['mean', 'min', 'max'],
    'Position_Y': ['mean', 'min', 'max'],
    'Vehicle_ID': 'count',
    'Speed': 'mean'
}).reset_index()
zone_properties.columns = ['Zone', 'Centroid_X', 'Min_X', 'Max_X', 'Centroid_Y', 'Min_Y', 'Max_Y', 'Vehicle_Count', 'Average_Speed']
print(zone_properties)

# RSU and Base Station Deployment
def deploy_rsu(location):
    print(f"Deploying RSU at location: {location}")

def deploy_base_station(location):
    print(f"Deploying Base Station at location: {location}")

rsu_locations = [(100, 200), (300, 400), (500, 600)]
base_station_location = (250, 350)
for loc in rsu_locations:
    deploy_rsu(loc)
deploy_base_station(base_station_location)

# Data Collection and Initial Content Precaching
def initial_precaching(data):
    precached_content = data[['Time', 'Content_Request_Size']].groupby('Time').sum().reset_index()
    print("Initial Content Precaching based on historical data:")
    print(precached_content.head())

initial_precaching(data)

# Intelligent Content Caching Mechanism
def intelligent_content_caching(predictions, actual):
    threshold = 0.1
    hits = np.abs(predictions - actual) <= threshold * actual
    hit_ratio = np.sum(hits) / len(actual)
    print(f"Cache Hit Ratio: {hit_ratio * 100:.2f}%")

intelligent_content_caching(y_pred, y_test_actual)

# Communication Mechanisms
def v2v_communication(vehicle_id):
    print(f"Vehicle {vehicle_id} initiated V2V communication")

def v2i_communication(vehicle_id, rsu_id):
    print(f"Vehicle {vehicle_id} communicated with RSU {rsu_id}")

v2v_communication(vehicle_id=1)
v2i_communication(vehicle_id=1, rsu_id=1)

# ZoR and ZoF definitions and content placement optimization
def define_zor(vehicle_position):
    print(f"Zone of Relevance defined for vehicle at position {vehicle_position}")

def define_zof(vehicle_position):
    print(f"Zone of Forwarding defined for vehicle at position {vehicle_position}")

for pos in data[['Position_X', 'Position_Y']].values[:5]:
    define_zor(pos)
    define_zof(pos)

# Early Content Prediction and Precaching
def early_content_precaching(predictions, actual):
    prediction_diff = np.mean(np.abs(predictions - actual))
    print(f"Early Content Precaching Performance (mean absolute error): {prediction_diff:.2f}")

early_content_precaching(y_pred, y_test_actual)

# Performance Evaluation
def evaluate_system_performance(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"System Performance - MSE: {mse:.2f}, MAE: {mae:.2f}")

evaluate_system_performance(y_test_actual, y_pred)


Epoch 1/50
10/10 [==============================] - 5s 128ms/step - loss: 0.1292 - val_loss: 0.1026
Epoch 2/50
10/10 [==============================] - 0s 12ms/step - loss: 0.0985 - val_loss: 0.0898
Epoch 3/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0931 - val_loss: 0.0809
Epoch 4/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0843 - val_loss: 0.0716
Epoch 5/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0774 - val_loss: 0.0646
Epoch 6/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0721 - val_loss: 0.0602
Epoch 7/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0686 - val_loss: 0.0583
Epoch 8/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0656 - val_loss: 0.0576
Epoch 9/50
10/10 [==============================] - 0s 15ms/step - loss: 0.0638 - val_loss: 0.0578
Epoch 10/50
10/10 [==============================] - 0s 17ms/step - loss: 0.0614 - val_loss: 0.0576
Epoch 11

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
